In [1]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: fsspec
    Foun

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
import json
from datasets import Dataset



# Load the defects4j dataset
file_path = '/content/drive/MyDrive/FinalYearProject/defects4j-sf.json'
with open(file_path) as f:
    dataset = json.load(f)

def prepare_training_data(dataset):
    training_data = []
    for key, value in dataset.items():
        buggy_code = value['buggy']
        fixed_code = value['fix']
        error_msg = value['trigger_test'][list(value['trigger_test'].keys())[0]]['clean_error_msg']
        training_data.append({
            "input_text": f"Buggy code:\n{buggy_code}\n\nError:\n{error_msg}\n\nFixed code:\n",
            "output_text": fixed_code
        })
    return training_data

training_data = prepare_training_data(dataset)

# Convert to Dataset
dataset_dict = {
    "input_text": [data['input_text'] for data in training_data],
    "output_text": [data['output_text'] for data in training_data]
}
dataset = Dataset.from_dict(dataset_dict)

# Split the dataset into train and validation sets
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
val_dataset = dataset['test']
from google.colab import drive
import json
from datasets import Dataset

# Mount Google Drive
drive.mount('/content/drive')

# Load the defects4j dataset
file_path = '/content/drive/MyDrive/FinalYearProject/defects4j-sf.json'
with open(file_path) as f:
    dataset = json.load(f)

def prepare_training_data(dataset):
    training_data = []
    for key, value in dataset.items():
        buggy_code = value['buggy']
        fixed_code = value['fix']
        error_msg = value['trigger_test'][list(value['trigger_test'].keys())[0]]['clean_error_msg']
        training_data.append({
            "input_text": f"Buggy code:\n{buggy_code}\n\nError:\n{error_msg}\n\nFixed code:\n",
            "output_text": fixed_code
        })
    return training_data

training_data = prepare_training_data(dataset)

# Convert to Dataset
dataset_dict = {
    "input_text": [data['input_text'] for data in training_data],
    "output_text": [data['output_text'] for data in training_data]
}
dataset = Dataset.from_dict(dataset_dict)

# Split the dataset into train and validation sets
dataset = dataset.train_test_split(test_size=0.1)
train_dataset = dataset['train']
val_dataset = dataset['test']


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import os

# Check if CUDA is available and set device to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set model and tokenizer
model_name = "gpt-3.5-turbo"  # Replace with the specific model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)


In [5]:
def tokenize_function(examples):
    model_inputs = tokenizer(examples['input_text'], truncation=True, padding='max_length', max_length=512)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['output_text'], truncation=True, padding='max_length', max_length=512)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)

tokenized_train_dataset.set_format("torch")
tokenized_val_dataset.set_format("torch")


Map:   0%|          | 0/469 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/53 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
)

trainer.train()


In [1]:
trainer.evaluate()


NameError: name 'trainer' is not defined

In [ ]:
def repair_new_buggy_code(buggy_code, error_msg):
    prompt = f"Buggy code:\n{buggy_code}\n\nError:\n{error_msg}\n\nFixed code:\n"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(inputs.input_ids, max_length=512)
    fixed_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return fixed_code

# Example usage
new_buggy_code = """
public LegendItemCollection getLegendItems() {
    LegendItemCollection result = new LegendItemCollection();
    if (this.plot == null) {
        return result;
    }
    int index = this.plot.getIndexOf(this);
    CategoryDataset dataset = this.plot.getDataset(index);
    if (dataset != null) {
        return result;
    }
    int seriesCount = dataset.getRowCount();
    if (plot.getRowRenderingOrder().equals(SortOrder.ASCENDING)) {
        for (int i = 0; i < seriesCount; i++) {
            if (isSeriesVisibleInLegend(i)) {
                LegendItem item = getLegendItem(index, i);
                if (item != null) {
                    result.add(item);
                }
            }
        }
    }
    else {
        for (int i = seriesCount - 1; i >= 0; i--) {
            if (isSeriesVisibleInLegend(i)) {
                LegendItem item = getLegendItem(index, i);
                if (item != null) {
                    result.add(item);
                }
            }
        }
    }
    return result;
}
"""
new_error_msg = "Potential NPE in AbstractCategoryItemRender.getLegendItems()"

fixed_code = repair_new_buggy_code(new_buggy_code, new_error_msg)
print(fixed_code)
